In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
from TravelDashboard.data import preproc_flight_data, get_data, save_processed_data

In [3]:
# get path of all .csv files (history from API crawling)
all_files = glob.glob(os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('')))),"jonas_pieper","project","api_call","output", "*.csv"))

In [11]:
print("Load local databases ...")
# Load required databases for matching/ preprocessing
seat_data = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath('')),"TravelDashboard", "data","seats_p_aircraft_final.csv"),
                        index_col=0)

#seat_data = pd.read_csv("TravelDashboard/data/seats_p_aircraft_final.csv",
#                        index_col=0)
aircraftregister_US = pd.read_csv(os.path.join(
    os.path.dirname(os.path.abspath('')), "TravelDashboard", "data",
    "US_ReleasableAircraftRegister.csv"),
                                    index_col=0,
                                    low_memory=False)
opensky_DB = pd.read_csv(os.path.join(
    os.path.dirname(os.path.abspath('')), "TravelDashboard", "data",
    "OpenSky_AircraftDatabase.csv"))


Load local databases ...


In [18]:
# here I will have to implement the foor loop 
start = 1399
end = 2000

for i, file in enumerate(all_files[start:end]):
    print(i+start)
    # Define column names for dataframe
    col_names = [
        'icao24', 'callsign', 'origin_country', 'time_position',
        'last_contact', 'long', 'lat', 'baro_altitude', 'on_ground',
        'velocity', 'true_track', 'vertical_rate', 'sensors', 'geo_altitude',
        'squawk', 'spi', 'position_source', "drop", "time"
    ]

    #Load raw data
    print("Load raw .csv files ...")
    # Load dataframe
    flights = pd.read_csv(file, index_col=0, header=0)

    if flights.shape[1]==19:
        flights.columns=col_names
        flights.drop(columns=[
        "time_position", "last_contact", "sensors", "squawk", "spi",
        "position_source", "drop"], inplace=True)  # drop columns
    elif flights.shape[1]==18: 
        flights.columns=col_names = [
        'icao24', 'callsign', 'origin_country', 'time_position',
        'last_contact', 'long', 'lat', 'baro_altitude', 'on_ground',
        'velocity', 'true_track', 'vertical_rate', 'sensors', 'geo_altitude',
        'squawk', 'spi', 'position_source', "time"]
        flights.drop(columns=[
        "time_position", "last_contact", "sensors", "squawk", "spi",
        "position_source"], inplace=True)  # drop columns
    else:
        continue

    flights = flights.dropna(
        subset=['lat', "long", "icao24"])  #drop rows where lat or long or icao24 is NaN -> can not be used for further processing

    pflights_df_final = preproc_flight_data(flights, opensky_DB, seat_data, aircraftregister_US)

    #Set path to store data
    path = os.path.join(os.path.dirname(os.path.abspath("")), "raw_data", "preproc_data_jonas", "{}.csv".format(int(pflights_df_final.time.unique()[0])))

    # Store preprocessed DataFrame to csv
    pflights_df_final.to_csv(path)

    print('Final DataFrame stored as .csv under {}'.format(path))

1399
Load raw .csv files ...
Start preprocessing ...
Shape before filter_passenger_carriers: (6641, 12)
Shape before get_aircraft_data: (4349, 15)
Shape before merge_seat_data: (4345, 19)
Shape before merge_seat_data_US: (4345, 21)
Shape before impute_missing_seats: (4345, 20)
Shape before increase_capacity_cheap_airlines: (4345, 20)
Shape before rev_geocode: (4345, 20)
Preprocessing finished
Final DataFrame stored as .csv under /home/philipp_heitmann/TravelDashboard/raw_data/preproc_data_jonas/1653136680.csv
1400
Load raw .csv files ...
Start preprocessing ...
Shape before filter_passenger_carriers: (4885, 12)
Shape before get_aircraft_data: (3140, 15)
Shape before merge_seat_data: (3136, 19)
Shape before merge_seat_data_US: (3136, 21)
Shape before impute_missing_seats: (3136, 20)
Shape before increase_capacity_cheap_airlines: (3136, 20)
Shape before rev_geocode: (3136, 20)
Preprocessing finished
Final DataFrame stored as .csv under /home/philipp_heitmann/TravelDashboard/raw_data/prep

In [17]:
pd.read_csv(all_files[1389], index_col=0, header=0)

,time


In [48]:
# test reading stored files
# pd.read_csv("/home/philipp_heitmann/TravelDashboard/raw_data/preproc_data_jonas/1654467161.csv", index_col=0)

In [49]:
# pflights_df_final.shape